In [1]:
import random
from pathlib import Path
import thinc.extra.datasets

import spacy
from spacy.util import minibatch, compounding

In [2]:

def train(train_data, test_data, model=None, output_dir=None, n_iter=10, init_tok2vec=None):
    '''
    train_data: training data, in the form of (train_texts [string], train_labels [{'class': bool}])
    test_data: test data, in the form of (val_texts [string], val_labels [{'class': bool}])
    model: Spacy model name, default to None for blank model.
    output_dir: directory to save trained model. Default to None to not save model.
    n_iter: number of iterations. Default to 10.
    '''
    
    
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()

    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print(f"Loaded model '{model}'")
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe("textcat")
    
    # add label to text classifier
    labels = train_data[0][1]['cats'].keys()
    for label in labels:
        textcat.add_label(label)
    
    textcat.add_label('OTHER')
    # get names of other pipes to disable them during training
    pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        if init_tok2vec is not None:
            with init_tok2vec.open("rb") as file_:
                textcat.model.tok2vec.from_bytes(file_.read())
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        batch_sizes = compounding(4.0, 32.0, 1.001)
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)
            
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, test_data)
            print(
                "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                )
            )

    if output_dir is not None:
        with nlp.use_params(optimizer.averages):
            nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
        
    
    return nlp


def evaluate(tokenizer, textcat, test_data):
    
    docs = (tokenizer(text) for (text, label) in test_data)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    
    for i, doc in enumerate(textcat.pipe(docs)):
        
        targets = test_data[i][1]['cats']
        
        for label, score in doc.cats.items():
            
            if label not in targets:
                continue
            if label == 'OTHER':
                continue
            
            if score >= 0.5 and targets[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and targets[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and targets[label] < 0.5:
                tn += 1
            elif score < 0.5 and targets[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}



In [3]:
def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the train data for evaluation
    data, _ = thinc.extra.datasets.imdb()
    random.shuffle(data)
    data = data[-limit:]
    
    data = [ (text_label_tuple[0], {"cats": {"POSITIVE": text_label_tuple[1]==1}}) for text_label_tuple in data]

    split = int(len(data) * split)
    train = data[:split]
    test = data[split:]
    return train, test

In [ ]:
# def load_data2(split=0.8):
#     df = pd.read_csv('data.csv')
#     cats = ['hypertension']
#     data = []
#     for index, row in df.iterrows():
#         text = row.text
#         label = {}
#         for cat in cats:
#             cat_label = (row[cat] == 'Yes' or row[cat] == 'Maybe')
#             label[cat] = cat_label
#         data.append((text,label))
#     random.shuffle(data)
#     texts,labels = zip(*data)
#     split = int(len(data)*split)
#     train_data = (texts[:split], labels[:split])
#     test_data = (texts[split:], labels[split:])
#     return train_data, test_data

In [4]:
train_data, test_data = load_data()

In [5]:
nlp = train(train_data, test_data)

Created blank 'en' model
Training the model...
LOSS 	  P  	  R  	  F  
7.304	0.835	0.844	0.839
0.074	0.850	0.853	0.851
0.052	0.858	0.863	0.860
0.043	0.873	0.859	0.866
0.034	0.872	0.869	0.871
0.028	0.877	0.871	0.874
0.023	0.879	0.866	0.872
0.020	0.878	0.875	0.877
0.017	0.877	0.878	0.878
0.013	0.871	0.883	0.877


In [58]:
# test the trained model
doc = nlp("this movie rocks")
print(doc.cats)

{'POSITIVE': 1.0, 'OTHER': 1.0324796937481967e-15}
